In [10]:
import numpy as np
import matplotlib 
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pandas import Series,DataFrame
%matplotlib inline
matplotlib.style.use('ggplot')
import sklearn

from datetime import datetime
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 8
print(np.version.full_version)
print(matplotlib.__version__)
print(pd.__version__)
print(sklearn.__version__)

2019/05/02 00:11:36
1.16.2
3.0.3
0.24.2
0.20.3


In [11]:
fp_train = "./avazu-ctr-prediction/train.csv"

fp_train_col_counts = "./avazu-ctr-prediction/col_counts"

fp_train_f = "./avazu-ctr-prediction/train_f.csv"

fp_train_lb = "./avazu-ctr-prediction/train_f_lb"
fp_train_oh = "./avazu-ctr-prediction/train_f_oh"

In [12]:
import pickle  # to store temporary variable
cols_counts = pickle.load(open(fp_train_col_counts, 'rb'))

sum = 0
for k, v in cols_counts.items():
    sum +=  v.size
    print("{} : {}".format(k, v.size))

print("all features: {} ".format(sum))

hour : 240
C1 : 7
banner_pos : 7
site_id : 4737
site_domain : 7745
site_category : 26
app_id : 8552
app_domain : 559
app_category : 36
device_id : 2686408
device_ip : 6729486
device_model : 8251
device_type : 5
device_conn_type : 4
C14 : 2626
C15 : 8
C16 : 9
C17 : 435
C18 : 4
C19 : 68
C20 : 172
C21 : 60
all features: 9449445 


In [13]:
features_mask_org =  ['hour', 'C1','banner_pos', 'site_id', 'site_domain', 'site_category', 
                      'app_id','app_domain', 'app_category', 
                      'device_id','device_ip', 'device_model', 'device_type', 'device_conn_type', 
                      'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

features_mask =  ['C1','banner_pos', 
                  'site_id', 'site_category', 
                  'app_domain', 'app_category', 
                  'device_model', 'device_type', 'device_conn_type', 
                  'C15', 'C16',  
                  'C18', 'C19', 'C20', 'C21']

print(set(features_mask_org) - set(features_mask))
#features_mask = features_mask + ['day_week', 'hour_day']

features_unique = 'id'

target_mask = 'click'

## data reading
#date_parser = lambda x: pd.datetime.strptime(x, '%y%m%d%H')
def rawcount(filename):
    f = open(filename, 'rb')
    lines = 0
    buf_size = 1024 * 1024
    read_f = f.raw.read

    buf = read_f(buf_size)
    while buf:
        lines += buf.count(b'\n')
        buf = read_f(buf_size)

    return lines


{'site_domain', 'C14', 'C17', 'device_ip', 'hour', 'app_id', 'device_id'}


In [15]:
## and set the rest to 'other'
k = 69
col_index = {}
for col in features_mask:
    col_index[col] = cols_counts[col][0: k].index

sum = 0
for k, v in col_index.items():
    sum +=  v.size
    print("{} : {}".format(k, v.size))

print("all features: {} ".format(sum))

C1 : 7
banner_pos : 7
site_id : 69
site_category : 26
app_domain : 69
app_category : 36
device_model : 69
device_type : 5
device_conn_type : 4
C15 : 8
C16 : 9
C18 : 4
C19 : 68
C20 : 69
C21 : 60
all features: 510 


In [16]:

# df_train_org = pd.read_csv(fp_train, dtype = {'id': str}, chunksize = 1000000, iterator = True)
df_train_org = pd.read_csv(fp_train, chunksize = 5000000, iterator = True,
                           dtype={'id':str},
                           #parse_dates=['hour'],
                           #date_parser=date_parser
                          )
import os
os.remove(fp_train_f) if os.path.exists(fp_train_f) else None

In [17]:
import gc

print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
hd_flag = True  # add column names at 1-st row
sum = 0

for chunk in df_train_org:
    df = chunk.copy()
    del chunk
    sum +=  len(df.index)
    print("～{} : {}".format(sum, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    
    for col in features_mask:
        df[col] = df[col].astype('object')
        # assign all the rare variables as 'other'
        df.loc[~df[col].isin(col_index[col]), col] = 'other'
    with open(fp_train_f, 'a') as f:
        df.to_csv(f, columns = features_mask + ["id", "click"], header = hd_flag, index = False)
        
        
    hd_flag = False
    #df.head()
    del df
    gc.collect()
    
del df_train_org

gc.collect()

print("Over")

2019/05/02 00:13:08
～5000000 : 2019/05/02 00:13:24
～10000000 : 2019/05/02 00:15:07
～15000000 : 2019/05/02 00:16:50
～20000000 : 2019/05/02 00:18:33
～25000000 : 2019/05/02 00:20:13
～30000000 : 2019/05/02 00:21:54
～35000000 : 2019/05/02 00:23:35
～40000000 : 2019/05/02 00:25:15
～40428967 : 2019/05/02 00:26:44
Over


In [18]:
from sklearn.preprocessing import LabelEncoder
from dummyPy import OneHotEncoder
import gc

## 1.label encoding
lb_enc = {}
for col in features_mask:
    col_index[col] = np.append(col_index[col], 'other')

for k, v in col_index.items():
    print("{} : {}".format(k, v.size))    
    
for col in features_mask:
    lb_enc[col] = LabelEncoder()
    lb_enc[col].fit(col_index[col])
    
## store the label encoder
pickle.dump(lb_enc, open(fp_train_lb, 'wb'))

del lb_enc


gc.collect()


print("Over")


C1 : 8
banner_pos : 8
site_id : 70
site_category : 27
app_domain : 70
app_category : 37
device_model : 70
device_type : 6
device_conn_type : 5
C15 : 9
C16 : 10
C18 : 5
C19 : 69
C20 : 70
C21 : 61


0

In [20]:
## 2.one-hot encoding
import gc

print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

oh_enc = OneHotEncoder(features_mask)

df_train_f = pd.read_csv(fp_train_f, index_col=None, chunksize=5000000, iterator=True)

sum = 0
for chunk in df_train_f:
    sum +=  len(chunk.index)
    print("～{} : {}".format(sum, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    
    oh_enc.fit(chunk)
    
    del chunk
    
print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"))    
## store the one-hot encoder
pickle.dump(oh_enc, open(fp_train_oh, 'wb'))

del oh_enc

gc.collect()

print("Over")

2019/05/02 00:29:38
2019/05/02 00:29:58
2019/05/02 00:30:18
2019/05/02 00:30:38
2019/05/02 00:30:57
2019/05/02 00:31:17
2019/05/02 00:31:37
2019/05/02 00:31:59
2019/05/02 00:32:19
2019/05/02 00:32:21
Over


In [21]:
## data reading
df = pd.read_csv(fp_train_f, nrows = 6,
                 dtype={'id':str}, 
                           #parse_dates=['hour'],
                           #date_parser=date_parser
                          )
df

,C1,banner_pos,site_id,site_category,app_domain,app_category,device_model,device_type,device_conn_type,C15,C16,C18,C19,C20,C21,id,click
0,1005,0,1fbe01fe,28905ebd,7801e8d9,07d7df22,other,1,2,320,50,0,35,-1,79,1000009418151094273,0
1,1005,0,1fbe01fe,28905ebd,7801e8d9,07d7df22,711ee120,1,0,320,50,0,35,100084,79,10000169349117863715,0
2,1005,0,1fbe01fe,28905ebd,7801e8d9,07d7df22,8a4875bd,1,0,320,50,0,35,100084,79,10000371904215119486,0
3,1005,0,1fbe01fe,28905ebd,7801e8d9,07d7df22,6332421a,1,0,320,50,0,35,100084,79,10000640724480838376,0
4,1005,1,other,0569f928,7801e8d9,07d7df22,779d90c2,1,0,320,50,0,35,-1,157,10000679056417042096,0
5,1005,0,d6137915,f028772b,7801e8d9,07d7df22,8a4875bd,1,0,320,50,0,431,100077,117,10000720757801103869,0


In [22]:
print(rawcount(fp_train_f))

40428968
